# Ejercicio 8. Consulta datos avtuales en API de AEMET

---

## **Objetivo:**
Desarrollar un script en Python que realice una consulta cada **15 minutos** a la **API de AEMET** para recopilar datos meteorológicos actuales de la estación `'C449C'` (Puerto de la Cruz, Tenerife) y los almacene en un archivo **CSV**.

---

## **Requisitos previos:**
1. **API Key de AEMET**:
   - Regístrate en [AEMET OpenData](https://opendata.aemet.es) y obtén tu API Key.
   
2. **Instalar librerías necesarias**:

In [ ]:
%%bash
pip install requests pandas

---

## **Paso 1: Código del Script**

Guarda este script en un archivo llamado, por ejemplo, `aemet_query.py`:

In [ ]:
import os
import requests
import pandas as pd
from datetime import datetime

# Parámetros para la consulta
AEMET_API_KEY = "TU_CLAVE_API"  # Reemplaza con tu API Key
ESTACION = "C449C"  # Código de la estación Puerto de la Cruz
URL = f"https://opendata.aemet.es/opendata/api/observacion/convencional/datos/estacion/{ESTACION}"

# Función para realizar la consulta y guardar los datos
def consulta_aemet():
    # Encabezados para la autenticación
    headers = {"api_key": AEMET_API_KEY}
    response = requests.get(URL, headers=headers)
    
    if response.status_code == 200:
        # URL de los datos (necesitamos hacer una segunda consulta)
        data_url = response.json().get("datos", None)
        if data_url:
            data_response = requests.get(data_url)
            if data_response.status_code == 200:
                data = data_response.json()
                # Procesar los datos
                registros = []
                for entry in data:
                    registro = {
                        "Fecha": entry.get("fint"),
                        "temperaturw": entry.get("ta"),
                        "humidity": entry.get("hr"),
                        "wind_speed": entry.get("vv"),
                        "wind_dir": entry.get("dv"),
                        "precipitation": entry.get("prec"),
                        "pressure": entry.get("pres"),
                        "wind_gusts": entry.get("vmax")
                    }
                    registros.append(registro)
                # Crear un DataFrame
                df = pd.DataFrame(registros)
                df['Fecha'] = pd.to_datetime(df['Fecha'])
                # Guardar en un archivo CSV
                archivo_csv = "data/datos_aemet_"+ESTACION+".csv"
                if not os.path.exists(archivo_csv):
                    df.to_csv(archivo_csv, index=False)
                else:    
                    # Leemos datos ya alamacenados
                    in_df = pd.read_csv(archivo_csv)
                    # Buscamos último registro de fecha disponible
                    ultima_fecha = pd.to_datetime(in_df.iloc[-1]['Fecha'])
                    print(ultima_fecha)
                    print(df)
                    #Filtramos solo datos nuevos
                    df = df[df['Fecha'] > ultima_fecha]
                    # Guardamos a final del archivo
                    df.to_csv(archivo_csv, mode="a", index=False, header=not pd.io.common.file_exists(archivo_csv))
                print(f"Datos guardados en {archivo_csv}")
            else:
                print(f"Error al obtener datos de {data_url}: {data_response.status_code}")
        else:
            print("No se pudo obtener la URL de los datos.")
    else:
        print(f"Error en la consulta a la API de AEMET: {response.status_code}")

# Ejecutar la consulta
if __name__ == "__main__":
    consulta_aemet()

---

## **Paso 2: Automatización para la Ejecución Cada 15 Minutos**

### En Linux/macOS:
1. **Abre el crontab** para editar:
   ```bash
   crontab -e
   ```

2. **Añade la siguiente línea** para ejecutar el script cada 15 minutos:
   ```bash
   */15 * * * * /usr/bin/python3 /ruta/a/tu/aemet_query.py
   ```

### En Windows:
1. Abre el **Programador de tareas**.
2. Crea una nueva tarea:
   - Configura el programa a ejecutar como:
     ```bash
     python
     ```
   - Especifica la ruta del archivo `aemet_query.py` como argumento.
3. Configura la tarea para ejecutarse cada 15 minutos.

---

## **Paso 3: Contenido del CSV Generado**

El archivo **`datos_aemet_C449C.csv`** tendrá el siguiente formato:

| Fecha y Hora       | Temperatura (°C) | Humedad (%) | Velocidad Viento (m/s) | Dirección Viento (°) | Precipitación (mm) | Presión (hPa) | Racha Máxima (m/s) |
|--------------------|------------------|-------------|-------------------------|-----------------------|--------------------|---------------|--------------------|
| 2024-11-01T15:00  | 22.5             | 65          | 3.1                     | 180                   | 0.0                | 1012          | 5.5                |
| 2024-11-01T15:15  | 22.3             | 66          | 3.2                     | 185                   | 0.0                | 1011          | 5.7                |

---

## **Resultados Esperados**

- Un archivo CSV actualizado cada 15 minutos con datos actuales de la estación `'C449C'`.
- Una base de datos histórica útil para realizar análisis meteorológicos y visualizaciones.
